# Notebook for testing showCamera.
Initially written by Michael Reuter.\
Modified by Craig Lage - 24-Sep-21\
Testing with AuxTel data from 2021-10-06.

In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.display as afwDisplay
import lsst.daf.butler as dafButler #Gen3 butler

In [2]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image.  Just bias subtraction and gain correction"""
    oim = camGeomUtils.rawCallback(im, ccd, imageSource,
                                   subtractBias=True, correctGain=True)
    return oim

In [3]:
# Instantiate the Gen3 butler
dataPath = "/repo/LATISS"
instrument = "LATISS"
butler = dafButler.Butler(dataPath, 
                          collections=["LATISS/raw/all", "LATISS/calib/unbounded"],
                          instrument=instrument)

In [27]:
day_obs = 20211007
seq_num = 122

raftName = "RXX"
dataId = {"instrument": instrument, "detector.raft": raftName, "detector": 0,
          "exposure.day_obs": day_obs, "exposure.seq_num": seq_num}

In [5]:
# camera has the info necessary to assemble the CCD
camera = butler.get('camera', instrument=instrument)
# Print the metadata just as a check.
metadata = butler.get('raw.visitInfo', {**dataId})
print(metadata)

DataId specified explicit detector dimension value of 0 in addition to general record specifiers for it of {'raft': 'RXX'}.  Ignoring record information.


VisitInfo(exposureId=2021100700002, exposureTime=2, darkTime=2.32546, date=2021-10-07T17:42:23.981658432, UT1=nan, ERA=4.9176 rad, boresightRaDec=(nan, +nan), boresightAzAlt=(359.9950820987, +79.9941537364), boresightAirmass=nan, boresightRotAngle=4.88876 rad, rotType=0, observatory=-70.7494W, -30.2446N  2663, weather=Weather(nan, nan, nan), instrumentLabel='LATISS', id=2021100700002)


In [6]:
# Print out mean and sigma for each detector
# This is before bias subtraction and gain adjustment.

exp = butler.get('raw', {**dataId})
arr = arr = exp.image.array
print(arr.min(), arr.max(), arr.mean(), arr.std())

DataId specified explicit detector dimension value of 0 in addition to general record specifiers for it of {'raft': 'RXX'}.  Ignoring record information.
/data/lsstdata/base/auxtel/oods/gen3butler/raw/2021-10-07/AT_O_20211007_000002-R00S00.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))


10761.0 174181.0 14484.609 2528.88


In [29]:
# Instantiate the firefly display.
# This should open a new tab with Firefly
disp = afwDisplay.Display(0, "firefly")

In [28]:
# This is bias subtracted and gain adjusted (see myCallback above), so gives a decent image.
# Note that it is only ~ 200-300 counts after bias subtraction
disp.scale('linear', 'zscale')

dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, raft=raftName,
                                                       day_obs=day_obs, seq_num=seq_num,
                                                       verbose=True, callback=myCallback,
                                                       background=np.nan),
                              binSize=16, display=disp, overlay=False,
                              title="%d %d %s" % (day_obs, seq_num, dataType))

/data/lsstdata/base/auxtel/oods/gen3butler/raw/2021-10-07/AT_O_20211007_000122-R00S00.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
